In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# ChromeDriverManager().install()

In [3]:
data_dir = "/Users/pervin0527/upstage-ml-regression-ml11/data"
train_data = pd.read_csv(f"{data_dir}/train.csv")
test_data = pd.read_csv(f"{data_dir}/test.csv")

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118597 non-null  object 
 2   본번                      1118747 non-null  float64
 3   부번                      1118747 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [5]:
train_data.isnull().sum()

시군구                             0
번지                            225
본번                             75
부번                             75
아파트명                         2126
전용면적(㎡)                         0
계약년월                            0
계약일                             0
층                               0
건축년도                            0
도로명                             0
해제사유발생일                   1112839
등기신청일자                          0
거래유형                            0
중개사소재지                          0
k-단지분류(아파트,주상복합등등)         870691
k-전화번호                     870274
k-팩스번호                     872742
단지소개기존clob                1050240
k-세대타입(분양형태)               869563
k-관리방식                     869563
k-복도유형                     869890
k-난방방식                     869563
k-전체동수                     870630
k-전체세대수                    869563
k-건설사(시공사)                 871058
k-시행사                      871254
k-사용검사일-사용승인일              869696
k-연면적                      869563
k-주거전용면적      

In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9272 entries, 0 to 9271
Data columns (total 51 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   시군구                     9272 non-null   object 
 1   번지                      9270 non-null   object 
 2   본번                      9272 non-null   float64
 3   부번                      9272 non-null   float64
 4   아파트명                    9262 non-null   object 
 5   전용면적(㎡)                 9272 non-null   float64
 6   계약년월                    9272 non-null   int64  
 7   계약일                     9272 non-null   int64  
 8   층                       9272 non-null   int64  
 9   건축년도                    9272 non-null   int64  
 10  도로명                     9272 non-null   object 
 11  해제사유발생일                 212 non-null    float64
 12  등기신청일자                  9272 non-null   object 
 13  거래유형                    9272 non-null   object 
 14  중개사소재지                  9272 non-null   

In [7]:
test_data.isnull().sum()

시군구                          0
번지                           2
본번                           0
부번                           0
아파트명                        10
전용면적(㎡)                      0
계약년월                         0
계약일                          0
층                            0
건축년도                         0
도로명                          0
해제사유발생일                   9060
등기신청일자                       0
거래유형                         0
중개사소재지                       0
k-단지분류(아파트,주상복합등등)        6582
k-전화번호                    6576
k-팩스번호                    6606
단지소개기존clob                8718
k-세대타입(분양형태)              6562
k-관리방식                    6562
k-복도유형                    6564
k-난방방식                    6562
k-전체동수                    6577
k-전체세대수                   6562
k-건설사(시공사)                6579
k-시행사                     6580
k-사용검사일-사용승인일             6563
k-연면적                     6562
k-주거전용면적                  6562
k-관리비부과면적                 6562
k-전용면적별세대현황(60㎡이하)        6562
k-전용면적별세

In [8]:
train_data['is_test'] = 0
test_data['is_test'] = 1

data = pd.concat([train_data, test_data])
data['is_test'].value_counts() 

0    1118822
1       9272
Name: is_test, dtype: int64

In [9]:
missing_data = data[data[['번지', '본번', '부번']].isnull().any(axis=1)]
print(len(missing_data))

227


In [10]:
values = []
for i, (index, row) in enumerate(missing_data.iterrows()):
    city = row['시군구']
    road_name = row['도로명']

    value = ' '.join(city.split(' ')[:-1]) + " " + road_name
    values.append(value)

print(len(values))

values_set = set(values)
print(values_set)

227
{'서울특별시 서초구 헌릉로8길 45', '서울특별시 서초구 헌릉로8길 10-12'}


In [11]:
url = "https://www.naver.com/"
browser = webdriver.Chrome()
browser.get(url)

results = {}
for idx, value in enumerate(values_set):
    print(f"검색어: {value}")

    if idx == 0:
        search_element = browser.find_element(By.ID, "query")
    else:
        search_element = browser.find_element(By.ID, "nx_query")
        search_element.clear()
    
    search_element.send_keys(value)
    search_element.send_keys(Keys.RETURN)

    try:
        target = browser.find_element(By.CLASS_NAME, 'L9_SH').text
        splited = target.split(' ')
        main_number = splited[-1]

        if "-" in main_number:
            splited = main_number.split('-')
            main_number = float(splited[0])
            sub_number = float(splited[1])
        else:
            main_number = float(main_number)
            sub_number = 0.0

        print(f"Main number: {main_number}, Sub number: {sub_number}")
        results[value] = (main_number, sub_number)

        time.sleep(3)
    except Exception as e:
        print(f"크롤링 오류 발생: {e}")
        continue

검색어: 서울특별시 서초구 헌릉로8길 45
Main number: 384.0, Sub number: 0.0
검색어: 서울특별시 서초구 헌릉로8길 10-12
Main number: 557.0, Sub number: 0.0


In [12]:
for i, (index, row) in enumerate(missing_data.iterrows()):
    city = row['시군구']
    road_name = row['도로명']
    value = ' '.join(city.split(' ')[:-1]) + " " + road_name

    if value in results:
        main_number, sub_number = results[value]
        data.loc[index, '번지'] = f"{int(main_number)}-{int(sub_number)}" if sub_number != 0 else f"{int(main_number)}"
        data.loc[index, '본번'] = int(main_number)
        data.loc[index, '부번'] = int(sub_number)

In [21]:
train_df = data[data['is_test'] == 0]
test_df = data[data['is_test'] == 1]

train_df = train_df.drop(columns=['is_test'])
test_df = test_df.drop(columns=['is_test', 'target'])

print(train_df.shape, test_df.shape)

os.makedirs(f'{data_dir}/new', exist_ok=True)
train_df.to_csv(f'{data_dir}/new/new_train_ver1.csv', index=False) 
test_df.to_csv(f'{data_dir}/new/new_test_ver1.csv', index=False)

browser.quit()

(1118822, 52) (9272, 51)


In [22]:
print(train_df.shape, test_df.shape)

(1118822, 52) (9272, 51)


In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1118822 entries, 0 to 1118821
Data columns (total 52 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   시군구                     1118822 non-null  object 
 1   번지                      1118822 non-null  object 
 2   본번                      1118822 non-null  float64
 3   부번                      1118822 non-null  float64
 4   아파트명                    1116696 non-null  object 
 5   전용면적(㎡)                 1118822 non-null  float64
 6   계약년월                    1118822 non-null  int64  
 7   계약일                     1118822 non-null  int64  
 8   층                       1118822 non-null  int64  
 9   건축년도                    1118822 non-null  int64  
 10  도로명                     1118822 non-null  object 
 11  해제사유발생일                 5983 non-null     float64
 12  등기신청일자                  1118822 non-null  object 
 13  거래유형                    1118822 non-null  object 
 14  중개

In [24]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9272 entries, 0 to 9271
Data columns (total 51 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   시군구                     9272 non-null   object 
 1   번지                      9272 non-null   object 
 2   본번                      9272 non-null   float64
 3   부번                      9272 non-null   float64
 4   아파트명                    9262 non-null   object 
 5   전용면적(㎡)                 9272 non-null   float64
 6   계약년월                    9272 non-null   int64  
 7   계약일                     9272 non-null   int64  
 8   층                       9272 non-null   int64  
 9   건축년도                    9272 non-null   int64  
 10  도로명                     9272 non-null   object 
 11  해제사유발생일                 212 non-null    float64
 12  등기신청일자                  9272 non-null   object 
 13  거래유형                    9272 non-null   object 
 14  중개사소재지                  9272 non-null   